In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau
import numpy as np
import pandas as pd
import cv2
import os
import numpy as np
import pandas as pd
import os
import cv2
import pandas as pd
from collections import Counter

### Ý nghĩa:
* Tải tất cả các thư viện cần thiết cho phân tích, xử lý và huấn luyện mô hình.

### Đề xuất (Pipeline):
* Cell này nên được giữ ở đầu pipeline.

# 1. Phân tích Kích thước Ảnh Gốc

Kiểm tra xem các ảnh gốc có đồng nhất về kích thước không. *Đây là bước phân tích, không phải bước xử lý.*

In [ ]:

# Định nghĩa đường dẫn (từ cell 6 của file gốc)
TRAIN_PATH = '../input/chest-xray-pneumonia/chest_xray/train'
TEST_PATH = '../input/chest-xray-pneumonia/chest_xray/test'
VAL_PATH = '../input/chest-xray-pneumonia/chest_xray/val'
PATHS = {'train': TRAIN_PATH, 'test': TEST_PATH, 'val': VAL_PATH}
LABELS = ['PNEUMONIA', 'NORMAL']

def get_original_image_sizes(data_path):
    sizes = []
    if not os.path.exists(data_path):
        print(f"Đường dẫn không tồn tại: {data_path}")
        return Counter()
        
    for label in LABELS:
        path = os.path.join(data_path, label)
        if not os.path.exists(path):
            continue
            
        for img_file in os.listdir(path):
            try:
                img_path = os.path.join(path, img_file)
                # Chỉ đọc header để lấy kích thước, không đọc cả ảnh -> nhanh hơn
                img = cv2.imread(img_path)
                if img is not None:
                    sizes.append(img.shape[:2]) # (height, width)
                else:
                    print(f"Không thể đọc file: {img_path}")
            except Exception as e:
                print(f"Lỗi file {img_path}: {e}")
    return Counter(sizes)

for name, path in PATHS.items():
    print(f"--- Đang quét tập {name} ---")
    size_counts = get_original_image_sizes(path)
    if size_counts:
        print(f"Các kích thước ảnh tìm thấy: {len(size_counts)} kích thước khác nhau")
        print(size_counts.most_common(5)) # In 5 kích thước phổ biến nhất
    else:
        print("Không tìm thấy dữ liệu.")


### Ý nghĩa:
* Cell code trên quét các file ảnh gốc và thống kê tất cả các kích thước (height, width) của chúng.
* **Kết quả:** Rất có thể bạn sẽ thấy **rất nhiều kích thước khác nhau**. Điều này khẳng định rằng dữ liệu đầu vào không đồng nhất.

### Đề xuất (Pipeline):
* **Bắt buộc:** Phải có một bước **Resize** đồng nhất tất cả ảnh về một kích thước cố định trước khi đưa vào model.
* **Lựa chọn:**
    * `150x150` (như trong file gốc của bạn): Tốt, nhanh, nhưng có thể mất chi tiết.
    * `224x224` (Tiêu chuẩn): Kích thước phổ biến cho ResNet, VGG. Đây là lựa chọn an toàn.
    * `299x299` (Cho Inception) hoặc lớn hơn (Cho EfficientNet).

# 2. Phân tích Phân chia Dữ liệu (Train/Val/Test)

In [ ]:

def count_files(data_path):
    count = 0
    if not os.path.exists(data_path): return 0
    for label in LABELS:
        path = os.path.join(data_path, label)
        if not os.path.exists(path): continue
        count += len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
    return count

train_size = count_files(TRAIN_PATH)
test_size = count_files(TEST_PATH)
val_size = count_files(VAL_PATH)

print(f"Train: {train_size}, Test: {test_size}, Val: {val_size}")

sizes = [train_size, test_size, val_size]
split_labels = ['Train', 'Test', 'Validation']
colors = ['orange', 'lightblue', 'white']
explode = (0, 0, 0.1)

plt.figure(figsize=(6,6))
wedges, texts, autotexts = plt.pie(
    sizes, labels=split_labels, colors=colors, autopct='%1.1f%%',
    explode=explode, startangle=90, wedgeprops={'edgecolor':'black'}
)
plt.title('Tỷ lệ phân chia Train/Test/Validation', fontsize=16)
plt.legend(wedges, split_labels, title="Tập", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.axis('equal')
plt.show()


### Ý nghĩa:
* Biểu đồ tròn cho thấy tỷ lệ phân chia dữ liệu.
* **Phân tích:** Tập `Validation` chỉ chiếm `1.1%` (từ file gốc). Đây là một con số **cực kỳ nhỏ** và **rất nguy hiểm**. Đánh giá model trên một tập val nhỏ như vậy sẽ không đáng tin cậy, kết quả sẽ bị dao động (variance) rất lớn.

### Đề xuất (Pipeline):
* **Hành động:** Gộp tập `train` và `val` gốc lại thành một bộ dữ liệu lớn.
* Sau đó, chia lại (resplit) bộ dữ liệu lớn này thành 2 tập (hoặc 3 tập) mới theo tỷ lệ 80/20 (Train/Val) hoặc 70/20/10 (Train/Val/Test). Sử dụng `sklearn.model_selection.train_test_split` để chia.

# 3. Phân tích Cân bằng Lớp (Class Balance)

In [ ]:

def get_class_distribution(data_path):
    dist = {}
    for label in LABELS:
        path = os.path.join(data_path, label)
        if not os.path.exists(path): continue
        count = len([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))])
        dist[label] = count
    return dist

train_dist = get_class_distribution(TRAIN_PATH)
print(f"Phân bổ lớp (Train): {train_dist}")

plt.figure(figsize=(6,4))
ax = sns.barplot(x=list(train_dist.keys()), y=list(train_dist.values()), palette="Set2")
# (Phần code annotate từ cell 8 gốc)
for p in ax.patches:
    height = int(p.get_height())
    ax.annotate(f'{height}', 
                (p.get_x() + p.get_width() / 2., height + 50),  
                ha='center', va='bottom', fontsize=12, color='black', fontweight='bold')
plt.title("Class distribution (Train set)")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()


### Ý nghĩa:
* Biểu đồ cho thấy số lượng ảnh của mỗi lớp trong tập train.
* **Phân tích:** Dữ liệu bị **mất cân bằng (imbalanced)** nghiêm trọng. Lớp 'PNEUMONIA' có số lượng ảnh gấp khoảng 3 lần lớp 'NORMAL'.
* Điều này sẽ khiến model bị thiên vị, có xu hướng dự đoán 'PNEUMONIA' nhiều hơn, và có thể đạt độ chính xác cao giả tạo (vì nó đoán đúng phần lớn các ca 'PNEUMONIA' nhưng lại đoán sai nhiều ca 'NORMAL').

### Đề xuất (Pipeline):
* **Giải pháp 1 (Nên làm):** Sử dụng **Data Augmentation** (Tăng cường dữ liệu) nhiều hơn cho lớp 'NORMAL' (lớp thiểu số).
* **Giải pháp 2 (Nên làm):** Khi gọi `model.fit()`, hãy cung cấp tham số `class_weight`. Tính toán `class_weight` để 'trừng phạt' model nặng hơn khi nó dự đoán sai lớp 'NORMAL'.
* **Giải pháp 3 (Tùy chọn):** Áp dụng kỹ thuật Oversampling (ví dụ: SMOTE) cho lớp 'NORMAL' hoặc Undersampling cho lớp 'PNEUMONIA'.

# 4. Trực quan hóa Ảnh mẫu

(Phần này cần dữ liệu đã được tải, nên ta sẽ định nghĩa hàm tải trước)

# 5. Đề xuất Pipeline Xử lý Dữ liệu

*Các cell dưới đây là các bước xử lý dữ liệu, được tổ chức lại từ file gốc của bạn.*

## 5.1. Định nghĩa Hàm Tải & Resize Dữ liệu

In [ ]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150

def get_training_data(data_dir):
    images = []
    target = []
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                if img_arr is None:  # tránh lỗi resize NoneType
                    continue
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                images.append(resized_arr)
                target.append(class_num)
            except Exception as e:
                print(f"Lỗi khi xử lý ảnh {img}: {e}")
    
    return images, target

### Ý nghĩa:
* Đây là hàm chính để tải dữ liệu. Quan trọng là, hàm này **đọc ảnh (GRAYSCALE) và RESIZE về `150x150` ngay lập tức** (`img_size = 150`).

### Đề xuất (Pipeline):
* Giữ nguyên hàm này. Kích thước `150x150` là một lựa chọn hợp lý để bắt đầu.
* **Lưu ý:** Vì ảnh được đọc ở dạng `GRAYSCALE` (1 kênh), model ở cuối phải có `input_shape = (150, 150, 1)`.

## 5.2. Tải Dữ liệu vào Bộ nhớ

In [ ]:
# Gọi hàm
x_train, y_train = get_training_data('../input/chest-xray-pneumonia/chest_xray/train')
x_test, y_test   = get_training_data('../input/chest-xray-pneumonia/chest_xray/test')
x_val, y_val     = get_training_data('../input/chest-xray-pneumonia/chest_xray/val')

print(x_train.shape, y_train.shape)

### Ý nghĩa:
* Tải dữ liệu vào các biến `x_train`, `y_train`...
* **Lưu ý:** Như đã phân tích ở mục 2, tập `x_val`, `y_val` là **quá nhỏ**. Bạn nên cân nhắc gộp và chia lại.

## 5.3. Trực quan hóa (Sau khi tải dữ liệu)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(x_train[0], cmap='viridis')
plt.title(labels[y_train[0]])

plt.figure(figsize=(5,5))
plt.imshow(x_train[-1], cmap='viridis')
plt.title(labels[y_train[-1]])


In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

# lấy 5 ảnh PNEUMONIA
pneumonia_samples = x_train[y_train == 0][:5]
for i, ax in enumerate(axes[0]):
    ax.imshow(pneumonia_samples[i], cmap='gray')
    ax.set_title("PNEUMONIA")
    ax.axis('off')

# lấy 5 ảnh NORMAL
normal_samples = x_train[y_train == 1][:5]
for i, ax in enumerate(axes[1]):
    ax.imshow(normal_samples[i], cmap='gray')
    ax.set_title("NORMAL")
    ax.axis('off')

plt.suptitle("Random samples from each class", fontsize=16)
plt.show()


In [ ]:
plt.figure(figsize=(10,5))
for i, lab in enumerate(labels):
    pixels = x_train[y_train==i].ravel()
    sns.histplot(pixels, bins=50, kde=True, label=lab, alpha=0.6)
plt.legend()
plt.title("Pixel Intensity Distribution per Class (Train set)")
plt.show()

### Ý nghĩa:
* Trực quan hóa một vài ảnh mẫu và phân bổ pixel. Giúp kiểm tra xem dữ liệu có bị lỗi, nhiễu, hay có gì bất thường không.
* **Phân tích:** Ảnh X-quang phổi, có vẻ rõ ràng.

### Đề xuất (Pipeline):
* Giữ nguyên các bước trực quan hóa này để kiểm tra (sanity check).

## 5.4. Chuẩn hóa (Normalization) và Reshape

In [ ]:
# Convert sang numpy array
x_train = np.array(x_train)
y_train = np.array(y_train)

x_val = np.array(x_val)
y_val = np.array(y_val)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Normalize the data
x_train = x_train / 255.0
x_val   = x_val / 255.0
x_test  = x_test / 255.0

x_train = x_train.reshape(-1, img_size, img_size, 1)
x_val   = x_val.reshape(-1, img_size, img_size, 1)
x_test  = x_test.reshape(-1, img_size, img_size, 1)

### Ý nghĩa:
* **Cell 1 (np.array):** Chuyển list ảnh sang mảng NumPy để xử lý.
* **Cell 2 (Normalization):** Chia pixel cho 255.0 để đưa giá trị về khoảng `[0, 1]`. Đây là bước **bắt buộc** cho các mạng neural network.
* **Cell 3 (Reshape):** Thêm một chiều kênh (channel) vào cuối, đổi shape từ `(N, 150, 150)` thành `(N, 150, 150, 1)`. Đây là bước **bắt buộc** vì Keras Conv2D yêu cầu đầu vào 4D.

### Đề xuất (Pipeline):
* Gộp 3 bước này lại và giữ nguyên, đây là các bước chuẩn.

## 5.5. Tăng cường Dữ liệu (Data Augmentation)

In [ ]:
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


data_generator.fit(x_train)

### Ý nghĩa:
* Định nghĩa một `ImageDataGenerator` để tạo thêm dữ liệu huấn luyện bằng cách xoay, zoom, lật ảnh ngẫu nhiên. Giúp model tổng quát hóa tốt hơn và giảm overfitting.

### Đề xuất (Pipeline):
* **Quan trọng:** Đây là giải pháp tốt cho vấn đề mất cân bằng lớp (mục 3). Bạn có thể tạo 2 generator: 1 cho 'PNEUMONIA' với các phép biến đổi nhẹ, và 1 cho 'NORMAL' (lớp thiểu số) với các phép biến đổi mạnh hơn (xoay nhiều hơn, zoom nhiều hơn) để tạo ra nhiều dữ liệu 'NORMAL' đa dạng hơn.